# Flask 聊天机器人应用程序解释
本笔记本详细解释了一个基于 Flask 的聊天机器人应用程序，该应用程序集成了 SparkApi。

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import SparkApi
import os
import threading
import time
import re

app = Flask(__name__)
CORS(app)

# 配置环境变量
os.environ["IFLYTEK_SPARK_URL"] = "wss://xingchen-api.cn-huabei-1.xf-yun.com/v1.1/chat"
os.environ["IFLYTEK_SPARK_APP_ID"] = "27741adb"
os.environ["IFLYTEK_SPARK_API_KEY"] = "7bf569c16f9330e40b49fba1ac28383b"
os.environ["IFLYTEK_SPARK_API_SECRET"] = "ZTNhYzdhNzI4NTExMGI4MTJjZjAyNjlk"

# 用于配置大模型版本，默认“general/generalv2”
domain = "xspark13b6k"  # v2.0版本
appid = os.getenv("IFLYTEK_SPARK_APP_ID")
api_secret = os.getenv("IFLYTEK_SPARK_API_SECRET")
api_key = os.getenv("IFLYTEK_SPARK_API_KEY")
Spark_url = os.getenv("IFLYTEK_SPARK_URL")

text = []

def getText(role, content):
    jsoncon = {"role": role, "content": content}
    text.append(jsoncon)
    return text

def getlength(text):
    length = 0
    for content in text:
        length += len(content["content"])
    return length

def checklen(text):
    while getlength(text) > 8000:
        del text[0]
    return text

stored_question = ""
stored_answer = ""

@app.route('/chat', methods=['POST'])
def chat():
    global stored_question, stored_answer
    data = request.get_json()
    stored_question = data.get('message')
    
    # 等待答案生成
    while not stored_answer:
        time.sleep(0.1)
    
    response = stored_answer
    stored_answer = ""  # 重置答案
    return jsonify({'answer': response})

    def process_questions(stored_question, stored_answer, loop=True):
    # global stored_question, stored_answer
    while True:
        if stored_question:
            prompt = ("？。 上面输入的问题属于下面ABCDE的哪一类，请输出对应的字母。\n"
                      "A: 查询企业信息。例如:查询企业的所属城市、营业期限、成立日期、纳税人识别号、法定代表人人、实缴资本、所属行业、匹配状态、所属城市等等一系列和企业信息有关的问题。\n"
                      "B: 招投标中的政策、法律咨询相关问题。包括招标投标的一般规定、招标范围、招标程序、监督处理、领域规范。还有政府采购的一般规定、采购主体、政策功能、采购方式等相关问题。\n"
                      "C: 企业舆情问题。\n"
                      "D: 查询招标信息。\n"
                      "E: 其他 例如:日常生活中常见的问题，通用的问题，不符合招投标领域的。\n")
            
            Input = stored_question + prompt
            stored_answer = ""
            question = checklen(getText("user", Input))
            SparkApi.answer = ""
            print(stored_question)
            print("星火:", end="")
            SparkApi.main(appid, api_key, api_secret, Spark_url, domain, question)
            getText("assistant", SparkApi.answer)
            output = text

            # 提取 'assistant' 的 content 值，并只保留第一个字符
            # assistant_content = next((item['content'] for item in output if item['role'] == 'assistant'), "")
            # stored_answer = assistant_content[0] if assistant_content else ""
            match = re.search(r'[ABCDE]', SparkApi.answer)
            stored_answer = match.group(0) if match else ""
            if stored_answer == "E":
                stored_answer = (" 你的问题描述不够准确，请记得添加关键词。输入的问题应该属于下面ABCD的一类。\n"
                      "A: 查询企业信息。例如:查询企业的所属城市、营业期限、成立日期、纳税人识别号、法定代表人人、实缴资本、所属行业、匹配状态、所属城市等等一系列和企业信息有关的问题。\n"
                      "B: 招投标中的政策、法律咨询相关问题。包括招标投标的一般规定、招标范围、招标程序、监督处理、领域规范。还有政府采购的一般规定、采购主体、政策功能、采购方式等相关问题。\n"
                      "C: 企业舆情问题。\n"
                      "D: 查询招标信息。\n")
            stored_question = ""  # Reset the question after processing
        if loop == False:
            return stored_answer
            break
        time.sleep(1)  # Add a small delay to prevent busy-waiting

if __name__ == '__main__':
    threading.Thread(target=process_questions(stored_question, stored_answer), daemon=True).start()
    app.run(debug=True)


## 解释

### 导入库
```python
from flask import Flask, request, jsonify
from flask_cors import CORS
import SparkApi
import os
import threading
import time
import re
```
这些行导入了应用程序所需的库：
- 从 `flask` 包中导入 `Flask`、`request` 和 `jsonify` 以创建 Web 应用程序并处理 HTTP 请求和响应。
- 从 `flask_cors` 包中导入 `CORS` 以处理跨域资源共享 (CORS) 问题。
- `SparkApi` 用于与 Spark API 交互。
- `os` 用于处理环境变量。
- `threading`、`time` 和 `re` 分别用于线程、时间管理和正则表达式。

### 初始化 Flask 应用程序
```python
app = Flask(__name__)
CORS(app)
```
这段代码初始化了 Flask 应用程序并为应用程序启用了 CORS。

### 配置环境变量
```python
os.environ["IFLYTEK_SPARK_URL"] = "wss://xingchen-api.cn-huabei-1.xf-yun.com/v1.1/chat"
os.environ["IFLYTEK_SPARK_APP_ID"] = "27741adb"
os.environ["IFLYTEK_SPARK_API_KEY"] = "7bf569c16f9330e40b49fba1ac28383b"
os.environ["IFLYTEK_SPARK_API_SECRET"] = "ZTNhYzdhNzI4NTExMGI4MTJjZjAyNjlk"
```
这些行设置了 Spark API 所需的环境变量，包括 URL、App ID、API Key 和 API Secret。

### 设置模型配置
```python
domain = "xspark13b6k"  # v2.0版本
appid = os.getenv("IFLYTEK_SPARK_APP_ID")
api_secret = os.getenv("IFLYTEK_SPARK_API_SECRET")
api_key = os.getenv("IFLYTEK_SPARK_API_KEY")
Spark_url = os.getenv("IFLYTEK_SPARK_URL")
```
这些行检索环境变量并设置 Spark API 的模型配置。

### 辅助函数
```python
text = []

def getText(role, content):
    jsoncon = {"role": role, "content": content}
    text.append(jsoncon)
    return text

def getlength(text):
    length = 0
    for content in text:
        length += len(content["content"])
    return length

def checklen(text):
    while getlength(text) > 8000:
        del text[0]
    return text
```
这些函数帮助管理文本数据：
- `getText(role, content)`: 使用指定的角色和内容将新消息添加到文本列表中。
- `getlength(text)`: 计算文本列表中所有消息的总长度。
- `checklen(text)`: 确保消息的总长度不超过 8000 个字符，通过删除最旧的消息。

### 全局变量
```python
stored_question = ""
stored_answer = ""
```
这些全局变量存储当前的问题和答案。

### 聊天端点
```python
@app.route('/chat', methods=['POST'])
def chat():
    global stored_question, stored_answer
    data = request.get_json()
    stored_question = data.get('message')
    
    # 等待答案生成
    while not stored_answer:
        time.sleep(0.1)
    
    response = stored_answer
    stored_answer = ""  # 重置答案
    return jsonify({'answer': response})


### 这个端点处理 POST 请求：

- 从请求中获取 JSON 数据并提取消息。
- 等待答案生成。
-  返回生成的答案，并重置存储的答案。

#
#
#





### 这个函数用于处理存储的问题和答案。它接受三个参数：

- stored_question: 存储的问题。
- stored_answer: 存储的答案。
- loop: 一个布尔值，决定是否循环执行。
```python
def process_questions(stored_question, stored_answer, loop=True):



### 这个循环不断检查是否有新的问题需要处理。
```python
while True:
    if stored_question:


### 这段代码生成一个提示，要求用户将输入的问题分类为 A、B、C、D 或 E。
```python
prompt = ("？。 上面输入的问题属于下面ABCDE的哪一类，请输出对应的字母。\n"
          "A: 查询企业信息。例如:查询企业的所属城市、营业期限、成立日期、纳税人识别号、法定代表人人、实缴资本、所属行业、匹配状态、所属城市等等一系列和企业信息有关的问题。\n"
          "B: 招投标中的政策、法律咨询相关问题。包括招标投标的一般规定、招标范围、招标程序、监督处理、领域规范。还有政府采购的一般规定、采购主体、政策功能、采购方式等相关问题。\n"
          "C: 企业舆情问题。\n"
          "D: 查询招标信息。\n"
          "E: 其他 例如:日常生活中常见的问题，通用的问题，不符合招投标领域的。\n")


### 这段代码将用户的问题和提示组合在一起，调用 SparkApi 处理问题，并将答案存储在 text 列表中。
```python
Input = stored_question + prompt
stored_answer = ""
question = checklen(getText("user", Input))
SparkApi.answer = ""
print(stored_question)
print("星火:", end="")
SparkApi.main(appid, api_key, api_secret, Spark_url, domain, question)
getText("assistant", SparkApi.answer)
output = text


### 这段代码使用正则表达式从 SparkApi 的答案中提取分类字母。如果答案是 “E”，则提示用户问题描述不够准确，并要求重新输入。
```python
match = re.search(r'[ABCDE]', SparkApi.answer)
stored_answer = match.group(0) if match else ""
if stored_answer == "E":
    stored_answer = (" 你的问题描述不够准确，请记得添加关键词。输入的问题应该属于下面ABCD的一类。\n"
          "A: 查询企业信息。例如:查询企业的所属城市、营业期限、成立日期、纳税人识别号、法定代表人人、实缴资本、所属行业、匹配状态、所属城市等等一系列和企业信息有关的问题。\n"
          "B: 招投标中的政策、法律咨询相关问题。包括招标投标的一般规定、招标范围、招标程序、监督处理、领域规范。还有政府采购的一般规定、采购主体、政策功能、采购方式等相关问题。\n"
          "C: 企业舆情问题。\n"
          "D: 查询招标信息。\n")


### 处理完问题后，重置 stored_question。
```python
stored_question = ""  # Reset the question after processing


### 如果 loop 为 False，则返回答案并退出循环。否则，添加一个小延迟以防止忙等待。
### 这里是用于测试
```python
if loop == False:
    return stored_answer
    break
time.sleep(1)  # Add a small delay to prevent busy-waiting


### 这段代码启动一个后台线程来处理问题，并运行 Flask 应用程序。
```python
if __name__ == '__main__':
    threading.Thread(target=process_questions(stored_question, stored_answer), daemon=True).start()
    app.run(debug=True)
